In [6]:
import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
import ollama
import anthropic
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display

In [7]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyAh


In [8]:
# Connect to OpenAI, Anthropic and Google
# All 3 APIs are similar
MODEL = 'ollama3.2'
openai = OpenAI()

client = anthropic.Anthropic()

In [9]:
# websie class
class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [34]:
plans = Website("https://www.t-mobile.com/cell-phone-plans")
phones = Website("https://www.t-mobile.com/cell-phones?INTNAV=tNav%3ADevices")
deals = Website("https://www.t-mobile.com/offers?INTNAV=tNav%3ADeals%3AAll")
payment_arrangement = Website("https://www.t-mobile.com/support/account/payment-arrangement")
payment = Website("https://www.t-mobile.com/support/account/pay-your-bill")
autopay = Website("https://www.t-mobile.com/support/account/autopay")
set_up_manage_your_TMO_id = Website("https://www.t-mobile.com/support/account/set-up-and-manage-your-t-mobile-id")
international_roaming = Website("https://www.t-mobile.com/support/coverage/international-roaming-services")
international_roaming_checklist = Website("https://www.t-mobile.com/support/coverage/international-roaming-checklist")
wi_fi_calling = Website("https://www.t-mobile.com/support/coverage/wi-fi-calling-from-t-mobile")
in_flight_connection = Website("https://www.t-mobile.com/support/coverage/t-mobile-in-flight-connections-on-us")

In [35]:
system_prompt = (
    f"You are a professional AI assistant designed to assist customers with inquiries about T-Mobile services, "
    f"including plans, phones, deals, and payment arrangements. Your responses must be accurate, clear, and concise, "
    f"using the provided information as reference.\n\n"
    f"Available Information Sources:\n"
    f"- Plans: {plans.text}\n"
    f"- Phones: {phones.text}\n"
    f"- Deals: {deals.text}\n"
    f"- Payment arrangement: {payment_arrangement.text}\n"
    f"- Payment: {payment.text}\n"
    f"- AutoPay: {autopay.text}\n"
    f"- Set Up and Manage Your T-Mobile ID:** {set_up_manage_your_TMO_id.text}\n"
    f"- International Roaming Services: {international_roaming.text}\n"
    f"- International Roaming Checklist: {international_roaming_checklist.text}\n\n"
    f"- WI-FI Calling: {wi_fi_calling.text}\n\n"
    f"- In-Flight Connection: {in_flight_connection.text}\n\n"
    f"Guidelines for Response:\n"
    f"1. ALWAYS include the appropriate link(s) in your response.\n"
    f"2. If the customer mentions 'Agent', 'customer service', 'representative', or 'human', respond with:\n"
    f"   tell the customer you are going to transfer them to a live represantative\n"
    f"3. If the customer asks a question in Spanish\n"
    f"   tell the cusotmer you are going to transfer them to a live represantative\n"
    f"4. If the customer asks account-specific questions (e.g., accessing their account, PIN requests, or similar) however you can provide information about how to make a payment arrengment, making a payment, respond with:\n"
    f"   'I'm sorry, but I can't access your account. Please reach out to a representative for further assistance.'\n\n"
    f"5. If the customer asks about plan price, provide the price with autopay included, and mention it.\n\n"
    f"6. always ask the customer if they need extra help unless they say they are done and not need any help.\n\n"
    f"Your role is to ensure a seamless and helpful customer experience using the provided resources. Follow these guidelines closely."
)

In [29]:
user_prompt = "the customer question"


In [30]:
# See how this function creates exactly the format above
def messages_for(question):
    return [
        { "role": "system", "content": system_prompt},
        { "role": "user", "content": question}
    ]

In [31]:
# OpenAI answer
def ask_ai(question):
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages = messages_for(question),
        max_tokens=1000,
        stream=True
    )
    reply = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        reply += chunk.choices[0].delta.content or ''
        reply = reply.replace("```","").replace("markdown","")
        update_display(Markdown(reply), display_id=display_handle.display_id)

In [32]:
ask_ai("can i have an extention on my bill?")

Yes, you can set up a payment arrangement for your T-Mobile account. To be eligible, your account must be less than 30 days past the due date. You can initiate the arrangement through the T-Life app or by visiting your T-Mobile account online. 

If you'd like assistance with setting up a payment arrangement, please follow this path:

1. Open the T-Life app.
2. Select the "Manage" tab.
3. Choose "Set up a payment arrangement."
4. Follow the prompts to set your arrangement.

If you have any additional questions or need help, let me know!

In [33]:
ask_ai("I'm going to France, does t-mobile have coverage there?")

Yes, T-Mobile offers coverage in France as part of its international roaming services. You can enjoy high-speed data, unlimited texting, and low flat-rate calling while you're abroad. Be sure to check the specific rates and coverage details for France on T-Mobile’s [International Roaming website](https://www.t-mobile.com/coverage/roaming). 

If you need more help with your travel plans, feel free to ask!

In [36]:
ask_ai("Do i have to make any changed to my phone?")

To use T-Mobile Wi-Fi Calling, you may need to ensure that your device supports Wi-Fi Calling and that it is enabled in your settings. Additionally, you should set up your E911 address to ensure emergency services can locate you if needed.

If you are using T-Mobile In-Flight Connection, ensure your device is in Airplane mode with Wi-Fi turned on, and that Wi-Fi Calling is enabled.

For detailed steps on enabling these features, you can refer to the tutorials on the T-Mobile support page or let me know if you need more specific guidance!

In [40]:
ask_ai("do you cover cruise ship")

Yes, T-Mobile offers roaming services on selected cruise lines. However, keep in mind that charges may apply depending on the cruise line's network. To find rates while traveling at sea, you can check the International Roaming page on T-Mobile's website and look for information specific to your cruise line. Please note that use of roaming services on cruise ships is generally more expensive than on land, so it's a good idea to keep your phone in airplane mode when in port to avoid unexpected charges.

If you have any additional questions or need further assistance, feel free to ask!

In [41]:
ask_ai("Hola")

I'm sorry, but I can't assist you in Spanish. I will transfer you to a live representative who can help you.